# 📁 DAVID mk36

In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime

todo_items = []

# 공통 레이아웃
layout_input = widgets.Layout(width='100%')
layout_button = widgets.Layout(width='100%', height='40px')
layout_label = widgets.Layout(width='100%', display='flex', justify_content='flex-start')
layout_row = widgets.Layout(display='flex', flex_flow='row', justify_content='space-between')
layout_center = widgets.Layout(display='flex', justify_content='center', width='100%')
layout_progress = widgets.Layout(width='100%')

# 입력 필드
title_input = widgets.Text(placeholder='할 일 입력', layout=layout_input)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_input)
due_input = widgets.Text(placeholder='마감일 (YYYY-MM-DD)', layout=layout_input)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_input)

add_button = widgets.Button(description='추가', button_style='success', layout=layout_button)
save_btn = widgets.Button(description='저장', button_style='info', layout=widgets.Layout(width='49%', height='40px'))
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=widgets.Layout(width='49%', height='40px'))

overall_progress_bar = widgets.FloatProgress(value=0, min=0, max=100, bar_style='', layout=layout_progress)
overall_progress_label = widgets.Label(value="0%", layout=widgets.Layout(width='auto'))

todo_list_box = widgets.VBox()

def update_overall_progress():
    if todo_items:
        total = sum(item['progress'] for item in todo_items)
        avg = total / len(todo_items)
        overall_progress_bar.value = avg
        overall_progress_label.value = f"{int(avg)}%"
    else:
        overall_progress_bar.value = 0
        overall_progress_label.value = "0%"

def refresh_list():
    todo_list_box.children = []
    for i, item in enumerate(todo_items, start=1):
        number_label = widgets.HTML(f"<b># {i}. {item['title']}</b>")
        progress = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_progress, bar_style='')
        progress.bar_style = ''
        color_value = int((item['progress'] / 100) * 255)
        progress.style = {'bar_color': f"rgb({color_value},{color_value},{color_value})"}
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_progress)
        slider.observe(lambda change, index=i-1: update_progress(index, change), names='value')
        date_label = widgets.Label(value=f"마감일 {item['due']}", layout=layout_label)
        priority_label = widgets.Label(value=f"중요도 {item['priority']}", layout=layout_label)
        complete_btn = widgets.Button(description='완료✅', layout=widgets.Layout(width='49%'))
        delete_btn = widgets.Button(description='삭제❌', layout=widgets.Layout(width='49%'), button_style='danger')
        complete_btn.style.button_color = "#e0e0e0"
        delete_btn.style.button_color = "#e0e0e0"
        complete_btn.on_click(lambda _, index=i-1: complete_task(index))
        delete_btn.on_click(lambda _, index=i-1: delete_task(index))
        row1 = widgets.HBox([progress])
        row2 = widgets.HBox([slider, widgets.Label(f"{item['progress']}%")])
        row3 = widgets.HBox([date_label, priority_label])
        row4 = widgets.HBox([complete_btn, delete_btn])
        card = widgets.VBox([number_label, row1, row2, row3, row4])
        todo_list_box.children += (card,)

def update_progress(index, change):
    todo_items[index]['progress'] = change['new']
    refresh_list()
    update_overall_progress()

def complete_task(index):
    todo_items[index]['progress'] = 100
    refresh_list()
    update_overall_progress()

def delete_task(index):
    del todo_items[index]
    refresh_list()
    update_overall_progress()

def add_task(_):
    try:
        title = title_input.value.strip()
        progress = progress_input.value
        due = datetime.strptime(due_input.value.strip(), '%Y-%m-%d').date()
        priority = priority_input.value
        todo_items.append({'title': title, 'progress': progress, 'due': due, 'priority': priority})
        title_input.value = ''
        progress_input.value = 0
        due_input.value = ''
        priority_input.value = '보통'
        refresh_list()
        update_overall_progress()
    except Exception as e:
        print("입력 오류:", e)

def save_data(_):
    with open("david_todos.json", "w") as f:
        json.dump(todo_items, f, default=str)

def load_data(_):
    try:
        with open("david_todos.json", "r") as f:
            loaded = json.load(f)
            for item in loaded:
                item['due'] = datetime.strptime(item['due'], '%Y-%m-%d').date()
            todo_items.clear()
            todo_items.extend(loaded)
        refresh_list()
        update_overall_progress()
    except Exception as e:
        print("불러오기 실패:", e)

add_button.on_click(add_task)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# UI 출력
display(widgets.HTML("<h3 style='margin-bottom: 0'>📁 DAVID mk36</h3>"))
display(widgets.Label("할 일"))
display(title_input)
display(widgets.Label("진행률"))
display(progress_input)
display(widgets.Label("마감일"))
display(due_input)
display(widgets.Label("중요도"))
display(priority_input)
display(add_button)
display(widgets.HBox([save_btn, load_btn]))
display(widgets.HTML("<b style='text-align:center;'>전체 진행률</b>", layout=layout_center))
display(widgets.HBox([overall_progress_bar, overall_progress_label]))
display(todo_list_box)
